In [7]:
# We first load the necessary libraries, the dataset and reshape its dimensons to the minimum allowed by the VGG16 --> (48,48,3)
import tensorflow as tf
from keras import callbacks
from keras import optimizers
from keras.engine import Model
from keras.layers import Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.applications import VGG16
from keras.utils import to_categorical
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn

sn.set(font_scale=1.4)
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

IMAGE_SIZE = (48, 48)


def load_data():
    """
      Load the data:
    """

    class_names = ['Hyundai Solaris sedan', 'KIA Rio sedan', 'SKODA OCTAVIA sedan', 'Volkswagen Polo sedan', 'Volkswagen Tiguan']
  
    class_names_label = {class_name: i for i, class_name in enumerate(class_names)}
    print("class_names_label: ", class_names_label)

    nb_classes = len(class_names)
    print("nb_classes: ", nb_classes)

    datasets = ['/content/drive/My Drive/VTB']
    output = []

    # Iterate through training and test sets
    # for dataset in datasets:
    if datasets:
        dataset = datasets[0]

        images = []
        labels = []

        print("Loading {}".format(dataset))

        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):

            print("folder: ", folder)
            if folder == ".ipynb_checkpoints": #or folder == "SKODA OCTAVIA sedan":
                print("Пропускаем: ", folder)
                continue

            check_folder = ['Hyundai Solaris sedan', 'KIA Rio sedan', 'SKODA OCTAVIA sedan', 'Volkswagen Polo sedan', 'Volkswagen Tiguan']
            if not folder in check_folder:
                print("Пропускаем: ", folder)
                continue

            label = class_names_label[folder]
            print("label: ", label)

            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):

                if file == ".DS_Store":
                    print("Пропуск: ", file)
                    continue

                if not ".jpeg" in file and not ".png" in file and not ".jpg" in file:
                    print("Not .jpeg: ", file)
                    continue

                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                try:
                    # Open and resize the img
                    #image = cv2.imread(img_path)
                    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #image = cv2.resize(image, IMAGE_SIZE)
                    img_width, img_height = 48, 48
                    img = image.load_img(img_path, target_size=(img_width, img_height))
                    img = image.img_to_array(img)
                    img = np.expand_dims(img, axis=0)
                except Exception as error:
                    print("img:", img_path, error)
                    continue

                # Append the image and its corresponding label to the output
                images.append(img)
                labels.append(label)

        try:
            images = np.array(images, dtype='float32')
        except Exception as error:
            print("images np.array: ", error)

        try:
            labels = np.array(labels, dtype='int32')
        except Exception as error:
            print("images np.array: ", error)
        # output.append((images, labels))

    # return output
    return (images, labels)


input_shape = (48, 48, 3)

### NEW
#datagen = ImageDataGenerator(rotation_range=90)
#(X_train, Y_train) = load_data()
#it = datagen.flow_from_directory(X_train, Y_train)
#it = datagen.flow_from_directory(X_train, Y_train, labels="inferred", label_mode="int")
### NEW

(X_train, Y_train) = load_data()

#Y_train = to_categorical(y_train, 4)

# resize train set
X_train_resized = []
for img in X_train:
    X_train_resized.append(np.resize(img, input_shape) / 255)

X_train_resized = np.array(X_train_resized)
print(X_train_resized.shape)

# We build the base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# We freeze every layer in our base model so that they do not train, we want that our feature extractor stays as before --> transfer learning
for layer in base_model.layers:
    layer.trainable = False
    print('Layer ' + layer.name + ' frozen.')

# We take the last layer of our the model and add it to our classifier
last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)
x = Dense(5, activation='softmax', name='predictions')(x)

model = Model(base_model.input, x)

# We compile the model
# new new
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer=Adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# new
# model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy']) 
# old
#model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# We start the training
# new
epochs = 20
batch_size = 128
# old
# epochs = 10
# batch_size = 256
# We train it
# new 
# model.fit_generator(it, steps_per_epoch=313, batch_size=batch_size)

# old
model.fit(X_train_resized, Y_train, 
          epochs=epochs,
          batch_size=batch_size,
          validation_split=0.0 )


# Схраняем модель
model.save('/content/drive/My Drive/VTB_Models/new_model.h5')



  0%|          | 0/1002 [00:00<?, ?it/s]

class_names_label:  {'Hyundai Solaris sedan': 0, 'KIA Rio sedan': 1, 'SKODA OCTAVIA sedan': 2, 'Volkswagen Polo sedan': 3, 'Volkswagen Tiguan': 4}
nb_classes:  5
Loading /content/drive/My Drive/VTB
folder:  Hyundai Solaris sedan
label:  0



  0%|          | 5/1002 [00:00<01:01, 16.15it/s]/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "

 14%|█▎        | 136/1002 [00:05<00:28, 30.66it/s]

img: /content/drive/My Drive/VTB/Hyundai Solaris sedan/141.jpeg cannot identify image file <_io.BytesIO object at 0x7f7f2cc8e1a8>



 56%|█████▌    | 560/1002 [00:23<00:17, 25.05it/s]

...

img: /content/drive/My Drive/VTB/SKODA OCTAVIA sedan/978.jpeg cannot identify image file <_io.BytesIO object at 0x7f7f2cc8e1a8>



 56%|█████▌    | 560/1002 [00:17<00:15, 29.41it/s]

Пропуск:  .DS_Store



100%|██████████| 1002/1002 [00:31<00:00, 32.31it/s]


folder:  .ipynb_checkpoints
Пропускаем:  .ipynb_checkpoints
(4838, 48, 48, 3)
Layer input_6 frozen.
Layer block1_conv1 frozen.
Layer block1_conv2 frozen.
Layer block1_pool frozen.
Layer block2_conv1 frozen.
Layer block2_conv2 frozen.
Layer block2_pool frozen.
Layer block3_conv1 frozen.
Layer block3_conv2 frozen.
Layer block3_conv3 frozen.
Layer block3_pool frozen.
Layer block4_conv1 frozen.
Layer block4_conv2 frozen.
Layer block4_conv3 frozen.
Layer block4_pool frozen.
Layer block5_conv1 frozen.
Layer block5_conv2 frozen.
Layer block5_conv3 frozen.
Layer block5_pool frozen.
Epoch 1/20
38/38 [==============================] - 108s 3s/step - loss: 1.5525 - accuracy: 0.3061
Epoch 2/20
38/38 [==============================] - 109s 3s/step - loss: 1.3696 - accuracy: 0.4229
Epoch 3/20
38/38 [==============================] - 109s 3s/step - loss: 1.2835 - accuracy: 0.4779
Epoch 4/20
38/38 [==============================] - 109s 3s/step - loss: 1.2123 - accuracy: 0.5143
Epoch 5/20
38/38 [=====